In [1]:
PATH_IMAGES = "./data/imgs_v2/"
PATH_MODELS = "./models/"

EPOCHS = 80
BATCH_SIZE = 10

import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"

if not(os.path.isdir(PATH_MODELS)):
    os.mkdir(PATH_MODELS)

In [2]:
import tensorflow as tf
tf.config.set_soft_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(gpus[0], True)

tf.config.experimental.set_virtual_device_configuration(gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*14)])

#gpus = tf.config.experimental.list_physical_devices('GPU')
#if gpus:
#    try:
#        # Currently, memory growth needs to be the same across GPUs
#        for gpu in gpus:
#            tf.config.experimental.set_memory_growth(gpu, True)
#        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#    except RuntimeError as e:
#        # Memory growth must be set before GPUs have been initialized
#        print(e)

#import tensorflow_lattice as tfl
print(tf.__version__)
print(tf.keras.backend.image_data_format())
import scipy.io 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


from tqdm import tqdm
tqdm.pandas()

import cv2
import PIL

import random
import sys
import math

from datetime import datetime

#tf.debugging.set_log_device_placement(True)

def pinball_loss(y_true, y_pred, tau=.5):
    """Computes the pinball loss between `y_true` and `y_pred`.
    `loss = maximum(tau * (y_true - y_pred), (tau - 1) * (y_true - y_pred))`
    In the context of regression this, loss yields an estimator of the tau
    conditional quantile.
    See: https://en.wikipedia.org/wiki/Quantile_regression
    Usage:
    ```python
    loss = pinball_loss([0., 0., 1., 1.], [1., 1., 1., 0.], tau=.1)
    # loss = max(0.1 * (y_true - y_pred), (0.1 - 1) * (y_true - y_pred))
    #      = (0.9 + 0.9 + 0 + 0.1) / 4
    print('Loss: ', loss.numpy())  # Loss: 0.475
    ```
    Args:
      y_true: Ground truth values. shape = `[batch_size, d0, .. dN]`
      y_pred: The predicted values. shape = `[batch_size, d0, .. dN]`
      tau: (Optional) Float in [0, 1] or a tensor taking values in [0, 1] and
        shape = `[d0,..., dn]`.  It defines the slope of the pinball loss. In
        the context of quantile regression, the value of tau determines the
        conditional quantile level. When tau = 0.5, this amounts to l1
        regression, an estimator of the conditional median (0.5 quantile).
    Returns:
        pinball_loss: 1-D float `Tensor` with shape [batch_size].
    References:
      - https://en.wikipedia.org/wiki/Quantile_regression
      - https://projecteuclid.org/download/pdfview_1/euclid.bj/1297173840
    """
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)

    # broadcast the pinball slope along the batch dimension, and clip to
    # acceptable values
    tau = tf.expand_dims(tf.cast(tau, y_pred.dtype), 0)
    one = tf.cast(1, tau.dtype)

    delta_y = y_true - y_pred
    pinball = tf.math.maximum(tau * delta_y, (tau - one) * delta_y)
    return tf.reduce_mean(tf.keras.backend.batch_flatten(pinball), axis=-1)


class PinballLoss(tf.keras.losses.Loss):
    """Computes the pinball loss between `y_true` and `y_pred`.
    `loss = maximum(tau * (y_true - y_pred), (tau - 1) * (y_true - y_pred))`
    In the context of regression, this loss yields an estimator of the tau
    conditional quantile.
    See: https://en.wikipedia.org/wiki/Quantile_regression
    Usage:
    ```python
    pinball = tfa.losses.PinballLoss(tau=.1)
    loss = pinball([0., 0., 1., 1.], [1., 1., 1., 0.])
    # loss = max(0.1 * (y_true - y_pred), (0.1 - 1) * (y_true - y_pred))
    #      = (0.9 + 0.9 + 0 + 0.1) / 4
    print('Loss: ', loss.numpy())  # Loss: 0.475
    ```
    Usage with the `compile` API:
    ```python
    model = tf.keras.Model(inputs, outputs)
    model.compile('sgd', loss=tfa.losses.PinballLoss(tau=.1))
    ```
    Args:
      tau: (Optional) Float in [0, 1] or a tensor taking values in [0, 1] and
        shape = `[d0,..., dn]`.  It defines the slope of the pinball loss. In
        the context of quantile regression, the value of tau determines the
        conditional quantile level. When tau = 0.5, this amounts to l1
        regression, an estimator of the conditional median (0.5 quantile).
      reduction: (Optional) Type of `tf.keras.losses.Reduction` to apply to
        loss. Default value is `AUTO`. `AUTO` indicates that the reduction
        option will be determined by the usage context. For almost all cases
        this defaults to `SUM_OVER_BATCH_SIZE`.
        When used with `tf.distribute.Strategy`, outside of built-in training
        loops such as `tf.keras` `compile` and `fit`, using `AUTO` or
        `SUM_OVER_BATCH_SIZE` will raise an error. Please see
        https://www.tensorflow.org/alpha/tutorials/distribute/training_loops
        for more details on this.
      name: Optional name for the op.
    References:
      - https://en.wikipedia.org/wiki/Quantile_regression
      - https://projecteuclid.org/download/pdfview_1/euclid.bj/1297173840
    """

    def __init__(self,
                 tau=.5,
                 reduction=tf.keras.losses.Reduction.AUTO,
                 name='pinball_loss'):
        super(PinballLoss, self).__init__(reduction=reduction, name=name)
        self.tau = tau

    def call(self, y_true, y_pred):
        return pinball_loss(y_true, y_pred, self.tau)

    def get_config(self):
        config = {
            'tau': self.tau,
        }
        base_config = super(PinballLoss, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm


def loader(path):
    try:
        im = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        #image = image * 1./255
        #image = cv2.resize(image, (224,224)) # non need anymore they are all the same size now
        return PIL.Image.fromarray(im)
    except OSError:
        print("Error:", path)
        return Image.new("RGB", (224, 224), "white")
    


2.0.0
channels_last


In [3]:
df = pd.read_pickle("./data/metadata_v3.pkl")
df.loc[df.MissingCount > 0, "Split"] = "unused"
del df["Path"]
df = df.rename(columns={"PathNew":"Path"})
df[["Recording", "PersonIdentity", "Frame", "Split", "Gaze", "Path"]].to_pickle("./data/metadata_small_v2.pkl")
del df

In [4]:
%%time
df = pd.read_pickle("./data/metadata_small_v2.pkl")
df = df.sort_values(["Recording", "PersonIdentity", "Frame"])
df.index = list(range(len(df)))

# WARNING!
# This only works with lots of memory ~45GB, use the generator which reads every step from disk
# WARNING!
df["Image"] = df.Path.progress_apply(lambda x: loader(x))

100%|██████████| 169672/169672 [02:19<00:00, 1217.07it/s]

CPU times: user 1min 54s, sys: 26.3 s, total: 2min 20s
Wall time: 2min 19s


In [5]:
df.Split.value_counts()

train     110255
unused     24219
test       21072
val        14126
Name: Split, dtype: int64

In [ ]:
# Random test image
if True:
    fig, ax = plt.subplots(3,5, figsize=(18,8))
    for i in range (3):
        for j in range (5):
            path = df.sample().iloc[0].Path
            img = loader(path)
            enh_bri = PIL.ImageEnhance.Brightness(img)
            img = enh_bri.enhance(random.uniform(.8,1.6))
            enh_cont = PIL.ImageEnhance.Contrast(img)
            img = enh_cont.enhance(random.uniform(.8,1.4))
            #Image.Contrast(.8)
            #PIL.ImageEnhance.Brightness(.8)
            ax[i][j].imshow(img)

In [7]:
class GazeDataset(tf.data.Dataset):
    def _generator(split, num_samples):
        global df
        
        dataIndexList = list(df[df.Split == split.decode()].index)
        while True:
            random.shuffle(dataIndexList)
            for idx in range(0, len(dataIndexList)-num_samples, num_samples):
                lstImg = []
                lstGaze = []
                contrast = random.uniform(.8,1.4)
                brightness = random.uniform(.8,1.6) # We know the images are overall dark
                for sampleCounter in range(num_samples):
                    nextIdx = dataIndexList[idx + sampleCounter]
                    x = df.iloc[nextIdx - 3 : nextIdx + 4]
                    lst = []
                    for img in x.Image.to_list():
                        enh_bri = PIL.ImageEnhance.Brightness(img)
                        img = enh_bri.enhance(brightness)
                        enh_cont = PIL.ImageEnhance.Contrast(img)
                        img = enh_cont.enhance(contrast)
                        img = np.array(img)
                        img = img * 1./255
                        lst.append(img)
                    imgs = np.array(lst)
                    gaze = x.iloc[3].Gaze
                    lstImg.append(imgs)
                    lstGaze.append(gaze)
                yield np.stack(lstImg), np.stack(lstGaze)
    
    def __new__(cls, split, num_samples):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.float32, tf.float32),
            output_shapes=([None, 7, 224,224,3], [None, 2]),
            args=(split, num_samples,)
        )

In [8]:
import time
def benchmark(dataset, num_epochs=5):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        i = 0
        for sample in dataset:
            # Performing a training step
            print(datetime.now(), epoch_num, i)
            i = i + 1
            break
    tf.print("Execution time:", time.perf_counter() - start_time)
    
benchmark(GazeDataset("train", BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE))

2020-04-05 20:56:11.850287 0 0
2020-04-05 20:56:12.683167 1 0
2020-04-05 20:56:13.515941 2 0
2020-04-05 20:56:14.314047 3 0
2020-04-05 20:56:15.106786 4 0
Execution time: 4.084275763481855


In [9]:
def error_Yaw(y_pred, y_true):
    error = tf.math.abs(y_true[:,0] - y_pred[:,0])
    error = 180.0 * tf.math.reduce_mean(error) / math.pi
    return error

def error_Pitch(y_pred, y_true):
    error = tf.math.abs(y_true[:,1] - y_pred[:,1])
    error = 180.0 * tf.math.reduce_mean(error) / math.pi
    return error


In [ ]:
time = datetime.now()
timeStr = str(time).split(".")[0].replace(" ", "-").replace(":", "-")

#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():
resnet = tf.keras.applications.MobileNetV2()
resnet.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Input((7, 224, 224, 3)),
    tf.keras.layers.TimeDistributed(resnet),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(512)),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 128, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 64, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 64)),
    tf.keras.layers.Dense(2)
])
model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss=PinballLoss(), metrics=[error_Pitch, error_Yaw])



path = "%scrf.%s-gaze360{epoch:03d}-{val_loss:.4f}.hdf5." % (PATH_MODELS, time)
callbackSaver = tf.keras.callbacks.ModelCheckpoint(path, monitor='val_loss')
#model.fit_generator will be deprecated use fit instead
history = model.fit(GazeDataset("train", BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE),
                    steps_per_epoch=int(len(df[df.Split == "train"])/BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=GazeDataset("val", BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE),
                    validation_steps=int(len(df[df.Split == "val"])/BATCH_SIZE),
                    callbacks=[])
                   #workers=30,
                   #use_multiprocessing=False)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 7, 1000)           3538984   
_________________________________________________________________
time_distributed_3 (TimeDist (None, 7, 512)            512512    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 7, 128)            295424    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 4,445,994
Trainable params: 907,010
Non-trainable params: 3,538,984
_________________________________________________________________
Train for 11025 steps, validate for 1412 steps
Epoch 1/80
11025/11025 [==============================] -